# Remainder Data Collection

In [1]:
import pandas as pd

# Full dataset of age-sex cohort
all_area = pd.read_excel('../Data/historyData.xlsx', sheet_name = 'Sheet1')
all_area_list = all_area["SA3 Name"].tolist()
all_area_year = all_area["Year"].tolist()
all_area_name = all_area["SA3 Name"].tolist()
all_area_cpde = all_area["SA3 Code"].tolist()
all_area_pop = all_area.iloc[0:len(all_area_year), 9:]
all_area_pop["Year"] = all_area_year
all_area_pop["SA3 Name"] = all_area_name
all_area_pop["SA3 Code"] = all_area_cpde

# Find our remainder areas from the age-sex cohort dataframe
remainder_list = []
outsider_list = ["Cocos (Keeling) Islands", "Jervis Bay", "Christmas Island"]
for i in range(len(all_area["Year"])):
    if (all_area.iloc[i, 45] == ".." or all_area.iloc[i, 45] < 1000):
        if (all_area.iloc[i, 8] not in remainder_list and all_area.iloc[i, 8] not in outsider_list):
            remainder_list.append(all_area.iloc[i, 8])

# Record Remainder Areas Age-Sex Cohort Data
age_sex_record_list = []
for i in range(len(all_area_list)):
    if all_area_list[i] in remainder_list:
        age_sex_record_list.append(all_area.iloc[i, 9:46].tolist())

for i in range(len(age_sex_record_list)):
    for j in range(len(age_sex_record_list[i])):
        if age_sex_record_list[i][j] == "..":
            age_sex_record_list[i][j] = 0

area_col = []
year_col = []
for i in range(1991,2012):
    for j in range(len(remainder_list)):
        year_col.append(i)
        area_col.append(remainder_list[j])

# Save the age-sex cohort information of the remainder areas
df_as = pd.DataFrame(age_sex_record_list, columns = all_area.columns[9:46].tolist())
df_as.insert(loc = 0, column = 'Year', value = year_col)
df_as.insert(loc = 0, column = 'Area', value = area_col)
df_as.to_csv("../Data/remainder_agesex.csv", index = False)

# Save the age-sex cohort aggregated information of the remainder areas
total_list = []
year_list = []
for j in range(1991, 2012):
    total = 0
    for i in range(len(remainder_list)):
        total += df_as.iloc[i + (j - 1991) * len(remainder_list), 2:39]
    total_list.append(total)
    year_list.append(j)

df_tot = pd.DataFrame(total_list)
df_tot.insert(loc = 0, column = "Year", value = year_list)
df_tot.to_csv("../Data/remainder_total.csv", index = False)

# Collect True Data

### Collect Age-Sex True Value

In [2]:
import pandas as pd

# Full dataset of age-sex cohort
all_area = pd.read_excel('../Data/historyData.xlsx', sheet_name = 'Sheet1')
remainder = pd.read_csv("../Data/remainder_total.csv")
row_name = pd.read_csv("../VBA_9616/Output_CSV/AgeSexForecasts.csv")
all_area_list = all_area["SA3 Name"].tolist()
all_area_year = all_area["Year"].tolist()
all_area_name = all_area["SA3 Name"].tolist()
all_area_cpde = all_area["SA3 Code"].tolist()
all_area_pop = all_area.iloc[0:len(all_area_year), 9:]
all_area_pop["Year"] = all_area_year
all_area_pop["SA3 Name"] = all_area_name
all_area_pop["SA3 Code"] = all_area_cpde

# Find out areas data from the age-sex cohort dataframe
area_list = []
real_area_list = []
remainder_list = []
outsider_list = ["Cocos (Keeling) Islands", "Jervis Bay", "Christmas Island"]
for i in range(len(all_area["Year"])):
    if (all_area.iloc[i, 45] == ".." or all_area.iloc[i, 45] < 1000):
        if (all_area.iloc[i, 8] not in remainder_list and all_area.iloc[i, 8] not in outsider_list):
            remainder_list.append(all_area.iloc[i, 8])
    else:
        if all_area.iloc[i, 8] not in remainder_list and all_area.iloc[i, 8] not in outsider_list:
            area_agesex = (all_area.iloc[i, 9:(9+36+1)].tolist())
            real_area_list.append(area_agesex)
            area_agesex = area_agesex[18:36] + area_agesex[0:18] + [area_agesex[36]]
            area_list.append(area_agesex)

valid_area = int(len(area_list) / (2011 - 1991 + 1))
index = 0
remainder_index = 0
full_list = []
above_1000_list = []
real_list = []
for i in range(len(area_list)):
    if (index == valid_area):
        index = 0
        remainder_lst = remainder.iloc[remainder_index, 1:38].tolist()
        real_list.append(remainder_lst)
        remainder_lst = remainder_lst[18:36] + remainder_lst[0:18] + [remainder_lst[36]]
        full_list.append(remainder_lst)
        remainder_index += 1
        real_list.append(real_area_list[i])
        above_1000_list.append(real_area_list[i])
        full_list.append(area_list[i])
        index += 1
    else:
        real_list.append(real_area_list[i])
        above_1000_list.append(real_area_list[i])
        full_list.append(area_list[i])
        index += 1
remainder_lst = remainder.iloc[-1, 1:38].tolist()
real_list.append(remainder_lst)
remainder_lst = remainder_lst[18:36] + remainder_lst[0:18] + [remainder_lst[36]]
full_list.append(remainder_lst)

index = 0
year_list = []
each_year = []
all_area_num = valid_area + 1
for i in range(len(full_list)):
    if (index == all_area_num):
        year_list.append(each_year)
        index = 1
        each_year = []
        for j in range(len(full_list[i][0:36])):
            each_year.append(full_list[i][j])
    else:
        for j in range(len(full_list[i][0:36])):
            each_year.append(full_list[i][j])
        index += 1
year_list.append(each_year)

pd_true = pd.DataFrame(year_list)
pd_true = pd_true.transpose()
pd_true.columns = remainder["Year"].tolist()
pd_true.insert(loc = 0, column = "Age Group", value = row_name["Age group"].tolist())
pd_true.insert(loc = 0, column = "Sex", value = row_name["Sex"].tolist())
pd_true.insert(loc = 0, column = "Area name", value = row_name["Area name"].tolist())
pd_true.insert(loc = 0, column = "Code", value = row_name["Code"].tolist())
pd_true.insert(loc = 0, column = "No.", value = row_name["No."].tolist())
pd_true.to_csv("../Data/True/agesex_true.csv", index = False)

### Store the True Dataframe for LSTM Model (Above_1000 & Full)

In [3]:
# Create Column Name List for the True Dataframe
real_data_name = all_area.columns.tolist()
real_data_name = [real_data_name[0]] + real_data_name[7:len(real_data_name)]

# Record Areas with above 1000 Total Population
real_areaname_1000_list = []
real_code_1000_list = []
for i in range(21):
    real_areaname_1000_list += row_name["Area name"].unique().tolist()[0:(len(row_name["Area name"].unique().tolist()) - 1)]
    real_code_1000_list += row_name["Code"].unique().tolist()[0:(len(row_name["Code"].unique().tolist()) - 1)]

real_year_1000_list = []
for i in range(len(remainder["Year"].tolist())):
    for j in range(len(row_name["Area name"].unique().tolist()) - 1):
        real_year_1000_list.append(remainder["Year"].tolist()[i])

real_1000_dataframe = pd.DataFrame(above_1000_list)
real_1000_dataframe.insert(loc = 0, column = "SA3_Name", value = real_areaname_1000_list)
real_1000_dataframe.insert(loc = 0, column = "SA3_Code", value = real_code_1000_list)
real_1000_dataframe.insert(loc = 0, column = "Year", value = real_year_1000_list)
real_1000_dataframe.columns = real_data_name
real_1000_dataframe.to_csv("../Data/true_1000_fulldata.csv", index = False)

# Record Areas with above 1000 Total Population and the Aggregated Remainder Area
real_areaname_list = []
real_code_list = []
for i in range(21):
    real_areaname_list += row_name["Area name"].unique().tolist()
    real_code_list += row_name["Code"].unique().tolist()

real_year_list = []
for i in range(len(remainder["Year"].tolist())):
    for j in range(len(row_name["Area name"].unique().tolist())):
        real_year_list.append(remainder["Year"].tolist()[i])

real_dataframe = pd.DataFrame(real_list)
real_dataframe.insert(loc = 0, column = "SA3_Name", value = real_areaname_list)
real_dataframe.insert(loc = 0, column = "SA3_Code", value = real_code_list)
real_dataframe.insert(loc = 0, column = "Year", value = real_year_list)
real_dataframe.columns = real_data_name
real_dataframe.to_csv("../Data/true_fulldata.csv", index = False)

### Collect Total True Value

In [5]:
index = 0
year_list = []
each_year = []
for i in range(len(full_list)):
    if (index == all_area_num):
        year_list.append(each_year)
        index = 1
        each_year = []
        each_year.append(full_list[i][36])
    else:
        each_year.append(full_list[i][36])
        index += 1
year_list.append(each_year)
pd_true = pd.DataFrame(year_list)
pd_true = pd_true.transpose()
pd_true.columns = remainder["Year"].tolist()
pd_true.to_csv("../Data/True/total_true.csv")